In [1]:
%load_ext autoreload
%autoreload 2

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

import os
import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

DEVICE = 'cuda:0'

In [2]:
from qlib import QuantizedLlamaForCausalLM

config = AutoConfig.from_pretrained("/home/msst/repo/Quantization/configs/Llama-3.2-1B_lsq3bit128gs.json")
q_model = QuantizedLlamaForCausalLM(config).to(torch.bfloat16)

LOL, Im quantized llama!


In [3]:
fp_model_path = "/home/msst/repo/Quantization/ml/llm/pretrained_models/Llama-3.2-1B"

fp_model = AutoModelForCausalLM.from_pretrained(
	fp_model_path,
	torch_dtype="auto"
)
tokenizer = AutoTokenizer.from_pretrained(
	fp_model_path,
)

with torch.no_grad():
	for q_module_name, q_module in q_model.named_modules():
		fp_module = fp_model.get_submodule(q_module_name)        
		if isinstance(q_module, qlib.LSQLinear):
			q_module.configure(fp_module.weight.data)
		elif hasattr(q_module, 'weight'):
			q_module.weight.copy_(fp_module.weight.data)

In [4]:
path_to_save = "/home/msst/repo/Quantization/logs/Llama-3.2-1B/Llama-3.2-1B_lsq3bit128gs_init"
q_model.to(torch.bfloat16).save_pretrained(path_to_save)
tokenizer.save_pretrained(path_to_save)

('/home/msst/repo/Quantization/logs/Llama-3.2-1B/Llama-3.2-1B_lsq3bit128gs_init/tokenizer_config.json',
 '/home/msst/repo/Quantization/logs/Llama-3.2-1B/Llama-3.2-1B_lsq3bit128gs_init/special_tokens_map.json',
 '/home/msst/repo/Quantization/logs/Llama-3.2-1B/Llama-3.2-1B_lsq3bit128gs_init/tokenizer.json')

In [5]:
w_q = q_model.get_decoder().layers[0].self_attn.q_proj.weight

In [6]:
w = fp_model.get_decoder().layers[0].self_attn.q_proj.weight